In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

In [2]:
result = pd.read_csv('result.csv', index_col=0)
da = pd.read_csv('DaycaresAnnotations_original.csv')
result['detection'] = 'daycare'
da['detection'] = 'daycare'

In [3]:
result.head(5)

,file,conf,x,y,xmin,ymin,xmax,ymax,width,height,detection
0,11926_20161008.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
1,11926_20170331.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
2,11946_20140215.jpg,0.597667,747.607422,422.410187,670.337044,339.748055,824.8778,505.072319,154.540756,165.324265,daycare
3,12645_20160829.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
4,12650_20120928.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare


In [4]:
ground_truth = []
for r in range(len(da)):
    if da.loc[r, 'region_shape_attributes'] != "{}":
        filename = da.loc[r, 'filename']
        rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])
        # x-min
        xmin = int(rect['x'])
        # x-max
        xmax = int(rect['x'] + rect['width'])
        # y-min
        ymin = int(rect['y'])
        # y-max
        ymax = int(rect['y'] + rect['height'])
        detection = da.loc[r, 'detection']
        ground_truth.append([filename, detection, xmin, ymax, xmax, ymin])
        
ground_truth = pd.DataFrame(np.array(ground_truth), columns = ['file', 'detection', 'xmin', 'ymax', 'xmax', 'ymin'])
positive_detection_result = result[['file', 'detection', 'conf', 'ymin', 'xmin','ymax','xmax']]

# detection_result = result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']]
# positive_detection_result = detection_result.loc[~((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]
# zero_detection_result = detection_result.loc[((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]


In [5]:
positive_detection_result.head(5)

,file,detection,conf,ymin,xmin,ymax,xmax
0,11926_20161008.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000
1,11926_20170331.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000
2,11946_20140215.jpg,daycare,0.597667,339.748055,670.337044,505.072319,824.8778
3,12645_20160829.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000
4,12650_20120928.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000


In [6]:
# pd.merge(df1, positive_detection_result, on='id', how='inner')

NameError: name 'df1' is not defined

In [8]:
positive_detection_result.head(5)

,file,detection,conf,ymin,xmin,ymax,xmax
0,11926_20161008.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000
1,11926_20170331.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000
2,11946_20140215.jpg,daycare,0.597667,339.748055,670.337044,505.072319,824.8778
3,12645_20160829.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000
4,12650_20120928.jpg,daycare,0.000000,0.000000,0.000000,0.000000,0.0000


In [9]:
positive_detection_files = positive_detection_result['file'].values
# zero_detection_result_files = zero_detection_result['file'].values
relevant_ground_truth = ground_truth[ground_truth['file'].isin(positive_detection_files)].drop_duplicates()


In [10]:
print(len(relevant_ground_truth), len(positive_detection_result))
print(set(relevant_ground_truth['file'].values) - set(positive_detection_result['file'].values))  
assert len(relevant_ground_truth) == len(positive_detection_result)

488 488
set()


In [11]:
positive_detection_result[['ymin', 'xmin', 'ymax', 'xmax']] = positive_detection_result[['ymin', 'xmin', 'ymax', 'xmax']].astype(int)
positive_detection_result.sort_values(by=['file']).head(5)

/Users/muhammadmuhaimin/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,file,detection,conf,ymin,xmin,ymax,xmax
0,11926_20161008.jpg,daycare,0.000000,0,0,0,0
1,11926_20170331.jpg,daycare,0.000000,0,0,0,0
2,11946_20140215.jpg,daycare,0.597667,339,670,505,824
3,12645_20160829.jpg,daycare,0.000000,0,0,0,0
4,12650_20120928.jpg,daycare,0.000000,0,0,0,0


In [15]:
relevant_ground_truth.sort_values(by=['file']).head(5)
relevant_ground_truth = relevant_ground_truth[['file', 'detection', 'ymin', 'xmin', 'ymax', 'xmax']]

In [16]:
def put_files(df, loc):
    values = [[value[0], np.array2string(value[1:], separator=' ')
       .replace('[', '').replace(']', '').replace("'", "").replace("\n", "")
     ]for value in df.values]
    for value in values:
        filepath_label = loc + value[0]
        filepath_label = filepath_label.replace('jpg', 'txt')
#         print(value[1])
        with open(filepath_label, 'w') as f:
            f.write("%s\n" % value[1])
put_files(positive_detection_result, './mAP_custom/input/detection-results/')
put_files(relevant_ground_truth, './mAP_custom/input/ground-truth/')